In [1]:
!pip install pandas
!pip install requests

import pandas as pd
import unicodedata
import datetime

You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
url_componentes = 'componentes-graduacao.csv'
url_unidades = 'http://dados.ufrn.br/dataset/da6451a5-1a59-4630-bdc2-97f6be4a59c2/resource/3f2e4e32-ef1a-4396-8037-cbc22a89d97f/download/unidades.csv'
url_docentes = 'http://dados.ufrn.br/dataset/8bf1a468-48ff-4f4d-95ee-b17b7a3a5592/resource/ff0a457e-76fa-4aca-ad99-48aebd7db070/download/docentes.csv'
url_avaliacoes = 'http://dados.ufrn.br/dataset/d5723d75-7e6e-4264-82aa-b96909b69f63/resource/7accd1d2-2793-460e-b98d-87a0679b9155/download/avaliacaodocencia.csv'

In [3]:
df_unidades = pd.read_csv(url_unidades, ';')
df_unidades = df_unidades[['id_unidade_responsavel', 'unidade_responsavel']]
df_unidades.rename(columns={'id_unidade_responsavel': 'id_unidade', 'unidade_responsavel': 'lotacao'}, inplace=True)
df_unidades = df_unidades[(df_unidades['lotacao'] == 'DEPARTAMENTO DE INFORMATICA E MATEMATICA APLICADA') | (df_unidades['lotacao'] == 'INSTITUTO METROPOLE DIGITAL')].drop_duplicates()
df_unidades.reset_index(inplace=True, drop=True)
df_unidades.head()

,id_unidade,lotacao
0,125,DEPARTAMENTO DE INFORMATICA E MATEMATICA APLICADA
1,6069,INSTITUTO METROPOLE DIGITAL


In [4]:
df_docentes = pd.read_csv(url_docentes, ';')
df_docentes.columns

Index([u'id_servidor', u'siape', u'nome', u'formacao',
       u'tipo_jornada_trabalho', u'vinculo', u'categoria', u'classe_funcional',
       u'id_unidade_lotacao', u'lotacao', u'admissao', u'Unnamed: 11'],
      dtype='object')

In [6]:
df_docentes.rename(columns={'id_servidor': 'id_docente', 'admissao': 'data_admissao', 'id_unidade_lotacao': 'unidade'}, inplace=True)
df_docentes = df_docentes.merge(df_unidades[['id_unidade']], left_on='unidade', right_on='id_unidade', how='inner')
#df_docentes.merge(df_avaliacoes[['docente']], left_on='id_docente', right_on='docente', how='inner').drop_duplicates('id_docente', inplace=True)
df_docentes['data_admissao'] = [ data.date() for data in pd.to_datetime(df_docentes['data_admissao'])]
df_docentes = df_docentes[['id_docente', 'nome', 'formacao', 'unidade', 'data_admissao']]
df_docentes.head()

,id_docente,nome,formacao,unidade,data_admissao
0,5755281,ADJA FERREIRA DE ANDRADE,DOUTORADO,6069,2012-11-01
1,5755819,ANDERSON PAIVA CRUZ,DOUTORADO,6069,2013-04-25
2,5757969,ANDRE LUIZ DA SILVA SOLINO,MESTRADO,6069,2016-01-19
3,5759298,ANNA GISELLE CAMARA DANTAS RIBEIRO RODRIGUES,DOUTORADO,6069,2017-05-26
4,3032089,APUENA VIEIRA GOMES,DOUTORADO,6069,2006-08-25


In [7]:
df_avaliacoes = pd.read_csv(url_avaliacoes, sep=';', usecols=['id_docente', 'id_turma', 'qtd_discentes', 'postura_profissional_media', 'postura_profissional_DP', 'atuacao_profissional_media', 'atuacao_profissional_DP'])
#df_avaliacoes = df_avaliacoes.merge(df_turmas[['id_turma']].astype(int), on='id_turma', how='inner')
df_avaliacoes = df_avaliacoes.merge(df_docentes[['id_docente']].astype(int), on='id_docente', how='inner')
df_avaliacoes.rename(columns={'postura_profissional_media': 'postura_profissional', 'atuacao_profissional_media': 'atuacao_profissional', 'id_docente': 'docente', 'id_turma': 'turma'}, inplace=True)
df_avaliacoes = df_avaliacoes[['docente', 'turma', 'qtd_discentes', 'postura_profissional', 'postura_profissional_DP', 'atuacao_profissional', 'atuacao_profissional_DP']]
df_avaliacoes.head()

,docente,turma,qtd_discentes,postura_profissional,postura_profissional_DP,atuacao_profissional,atuacao_profissional_DP
0,22764,1233659,17,9.09,1.57,9.00,1.60
1,22764,1233661,36,9.48,1.26,9.43,1.20
2,22764,1242644,34,9.41,1.04,8.86,1.73
3,22764,57566712,36,9.81,0.51,9.48,1.35
4,22764,57566818,14,9.69,0.48,8.13,2.44


In [8]:
turma_periodo = ['http://dados.ufrn.br/dataset/1938623d-fb07-41a4-a55a-1691f7c3b8b5/resource/5e77d066-d506-45eb-a21e-76aa79616fef/download/turmas-2017.1',
'http://dados.ufrn.br/dataset/1938623d-fb07-41a4-a55a-1691f7c3b8b5/resource/01fe7343-fdf0-4a67-b915-2386b7c2fecb/download/turmas-2017.2',
'http://dados.ufrn.br/dataset/1938623d-fb07-41a4-a55a-1691f7c3b8b5/resource/322d9977-ba15-47f1-8216-75a1ca78e197/download/turmas-2016.1.csv',
'http://dados.ufrn.br/dataset/1938623d-fb07-41a4-a55a-1691f7c3b8b5/resource/5e8e3228-7f22-40a2-9efd-561c44844567/download/turmas-2016.2.csv',
'http://dados.ufrn.br/dataset/1938623d-fb07-41a4-a55a-1691f7c3b8b5/resource/4d5aee5a-00b0-4ed6-a4be-59fa77a56797/download/turmas-2015.1.csv',
'http://dados.ufrn.br/dataset/1938623d-fb07-41a4-a55a-1691f7c3b8b5/resource/7c59621c-4a8b-49d4-b319-83cfea9bdf28/download/turmas-2015.2.csv',
'http://dados.ufrn.br/dataset/1938623d-fb07-41a4-a55a-1691f7c3b8b5/resource/e6e4144f-4042-4fdc-84e0-76e9ec27ae7c/download/turmas-2014.1.csv',
'http://dados.ufrn.br/dataset/1938623d-fb07-41a4-a55a-1691f7c3b8b5/resource/2c69547b-920f-4ec2-92c0-3fbc19512165/download/turmas-2014.2.csv',
'http://dados.ufrn.br/dataset/1938623d-fb07-41a4-a55a-1691f7c3b8b5/resource/0d96d930-8058-4def-9044-c3ae04c1f40c/download/turmas-2013.1.csv',
'http://dados.ufrn.br/dataset/1938623d-fb07-41a4-a55a-1691f7c3b8b5/resource/e7e3cf12-a29b-491a-a895-021a43819197/download/turmas-2013.2.csv',]

In [9]:
frames = [pd.read_csv(periodo, sep=';', usecols=['id_turma', 'ano', 'nivel_ensino', 'periodo', 'id_componente_curricular']) for periodo in turma_periodo]

In [10]:
df_turmas = pd.concat(frames)

In [11]:
df_turmas.rename(columns={'id_turma': 'id_turma', 'nivel_ensino': 'nivel', 'id_componente_curricular': 'componente'}, inplace=True)
df_turmas.drop_duplicates('id_turma', inplace=True)
df_turmas['periodo'] = df_turmas['periodo'].astype(int) 
df_turmas['componente'] = df_turmas['componente'].astype(int)
df_turmas = df_turmas.merge(df_avaliacoes[['turma']], left_on='id_turma', right_on='turma', how='inner')
df_turmas = df_turmas[['id_turma', 'componente', 'nivel', 'ano', 'periodo']]
df_turmas.head()

,id_turma,componente,nivel,ano,periodo
0,57595535,63609,GRADUAÇÃO,2017,1
1,57596647,2022854,GRADUAÇÃO,2017,1
2,57596648,2022855,GRADUAÇÃO,2017,1
3,57596649,21771,GRADUAÇÃO,2017,1
4,57596651,52866,GRADUAÇÃO,2017,1


In [ ]:
## APENAS GRADUAÇÃO, REMOVER CASO DESEJE TUDO
df_docentes = df_docentes[df_docentes['categoria'] == 'PROFESSOR DO MAGISTERIO SUPERIOR']
df_turmas = df_turmas[df_turmas['nivel'] == 'GRADUAÇÃO']

In [12]:
df_componentes = pd.read_csv(url_componentes, sep=',')
df_componentes.merge(df_turmas[['componente']], left_on='id_componente', right_on='componente', how='left').drop_duplicates('id_componente', inplace=True)
df_componentes = df_componentes[['id_componente', 'codigo', 'nivel', 'nome', 'unidade_responsavel']]

In [13]:
unidades_normalizadas = [unicodedata.normalize('NFKD', unicode(nome, "utf-8")).encode('ASCII', 'ignore') for nome in df_componentes['unidade_responsavel']]
series = pd.Series(data=unidades_normalizadas, index=df_componentes.index)
df_componentes['unidade_responsavel'] = series

In [14]:
df_componentes = df_componentes.merge(df_unidades[['id_unidade', 'lotacao']], left_on='unidade_responsavel', right_on='lotacao', how='inner')
df_componentes.rename(columns={'id_unidade': 'unidade'}, inplace=True)
df_componentes = df_componentes[['id_componente', 'codigo', 'nivel', 'nome', 'unidade']]
df_componentes.head()

,id_componente,codigo,nivel,nome,unidade
0,20007,DIM0072,G,TECNICAS DE ESPECIFICACAO FORMAL,125
1,21073,DIM0056,G,SOFTWARE BASICO,125
2,21083,DIM0346,G,GERENCIAMENTO E SEGURANCA EM REDES DE COMPUTAD...,125
3,21667,DIM0046,G,ALGORITMOS E TECNICAS DE PROGRAMACAO,125
4,21668,DIM0003,G,ORGANIZACAO DE COMPUTADORES,125


In [15]:
matriculas = [
    'http://dados.ufrn.br/dataset/c8650d55-3c5a-4787-a126-d28a4ef902a6/resource/55dfe713-ff7c-4fa8-8d1d-d4294a025bff/download/matricula-componente-20172',
    'http://dados.ufrn.br/dataset/c8650d55-3c5a-4787-a126-d28a4ef902a6/resource/79071c21-e32c-438f-b930-d1b6ccc02ec2/download/matricula-componente-20171',
    'http://dados.ufrn.br/dataset/c8650d55-3c5a-4787-a126-d28a4ef902a6/resource/f6179838-b619-4d7d-af9c-18c438b80dd4/download/matriculas-de-2016.2.csv',
    'http://dados.ufrn.br/dataset/c8650d55-3c5a-4787-a126-d28a4ef902a6/resource/4778d3ce-8898-46a8-a623-ee6a480a2980/download/matriculas-de-2016.1.csv',
    'http://dados.ufrn.br/dataset/c8650d55-3c5a-4787-a126-d28a4ef902a6/resource/baa6c8b4-2072-417f-b238-c028ccc8c14b/download/matriculas-de-2015.2.csv',
    'http://dados.ufrn.br/dataset/c8650d55-3c5a-4787-a126-d28a4ef902a6/resource/9e7ba1c2-f92d-4b9c-9e91-3b026ecdf913/download/matriculas-de-2015.1.csv',
    'http://dados.ufrn.br/dataset/c8650d55-3c5a-4787-a126-d28a4ef902a6/resource/e974792c-b557-470c-bf3d-ede7d5b5e6a6/download/matricula-componente-20142.csv',
    'http://dados.ufrn.br/dataset/c8650d55-3c5a-4787-a126-d28a4ef902a6/resource/7081446d-39f9-4374-ad0b-86ecab97e569/download/matricula-componente-20141.csv',
    'http://dados.ufrn.br/dataset/c8650d55-3c5a-4787-a126-d28a4ef902a6/resource/517ed5f6-f8a2-40fd-826b-6ed3388f6e88/download/matricula-componente-20132.csv',
    'http://dados.ufrn.br/dataset/c8650d55-3c5a-4787-a126-d28a4ef902a6/resource/674de4cc-1fc0-4314-9f04-a38f0e1f9225/download/matricula-componente-20131.csv',
]

In [16]:
dfs_matricula = [pd.read_csv(matricula, sep=';') for matricula in matriculas]

In [17]:
df_matriculas = pd.concat(dfs_matricula)

In [18]:
df_matriculas = df_matriculas[~df_matriculas['descricao'].isin(['CUMPRIU', 'INDEFERIDO', 'EXCLUIDA'])]

In [19]:
df_matriculas.replace('APROVADO POR NOTA', 'APROVADO', inplace=True)
df_matriculas.replace(['REPROVADO POR NOTA', 'REPROVADO POR FALTAS', 'REPROVADO POR MÉDIA E POR FALTAS', 'REPROVADO POR NOTA E FALTA', 'DESISTENCIA', 'TRANCADO', 'CANCELADO'], 'REPROVADO', inplace=True)

In [20]:
df_matriculas = df_matriculas.merge(df_turmas[['id_turma']].astype(int), on='id_turma', how='inner')
df_matriculas.head()

,Unnamed: 10,descricao,discente,faltas_unidade,id_curso,id_turma,media_final,nota,numero_total_faltas,reposicao,unidade
0,NaN,REPROVADO,bc390f3083818821c8a9ba693b7e9bf7,0.0,2000002.0,57595535,0.0,0.0,60.0,f,3.0
1,NaN,REPROVADO,bc390f3083818821c8a9ba693b7e9bf7,0.0,2000002.0,57595535,0.0,0.0,60.0,f,2.0
2,NaN,REPROVADO,bc390f3083818821c8a9ba693b7e9bf7,0.0,2000002.0,57595535,0.0,0.0,60.0,f,1.0
3,NaN,APROVADO,780f0ab3e40fcef14e189c9f4b58a512,0.0,2000002.0,57595535,9.1,8.3,5.0,f,3.0
4,NaN,APROVADO,780f0ab3e40fcef14e189c9f4b58a512,0.0,2000002.0,57595535,9.1,9.5,5.0,f,2.0


In [21]:
df_aprovados = df_matriculas[df_matriculas['descricao'] == 'APROVADO']
df_media_aprovados = df_aprovados.groupby('id_turma')['media_final'].mean().to_frame('media_aprovados').reset_index(level=0)
df_media_aprovados.head()

,id_turma,media_aprovados
0,1208017,6.983333
1,1208019,6.604348
2,1208040,7.058824
3,1208456,7.609184
4,1209057,5.912500


In [22]:
#media_turma = df_matriculas.groupby('id_turma')['media_final'].mean().to_frame('média').reset_index(level=0)
#media_turma.head()

In [23]:
df_porcentagem = df_matriculas.groupby('id_turma')['descricao'].value_counts(normalize=True).to_frame('aprovados')

In [24]:
df_porcentagem = df_porcentagem.reset_index(level=[0, 1])
df_porcentagem = df_porcentagem[~df_porcentagem['descricao'].isin(['REPROVADO'])]
df_porcentagem = df_porcentagem[['id_turma', 'aprovados']]
df_porcentagem.head()

,id_turma,aprovados
0,1208017,0.765957
2,1208019,0.570248
4,1208040,0.772727
6,1208456,0.788204
8,1209057,0.648649


In [25]:
df_avaliacoes = df_avaliacoes.merge(df_media_aprovados, left_on='turma', right_on='id_turma', how='inner')
df_avaliacoes = df_avaliacoes.merge(df_porcentagem, left_on='turma', right_on='id_turma', how='inner')
df_avaliacoes = df_avaliacoes[['docente', 'turma', 'qtd_discentes', 'postura_profissional', 'postura_profissional_DP', 'atuacao_profissional', 'atuacao_profissional_DP', 'media_aprovados', 'aprovados']]
df_avaliacoes.head()

,docente,turma,qtd_discentes,postura_profissional,postura_profissional_DP,atuacao_profissional,atuacao_profissional_DP,media_aprovados,aprovados
0,22764,1242644,34,9.41,1.04,8.86,1.73,7.603030,0.916667
1,22764,57566712,36,9.81,0.51,9.48,1.35,7.578788,0.860870
2,22764,57566818,14,9.69,0.48,8.13,2.44,7.458333,0.837209
3,23809,57566818,14,9.69,0.48,8.22,2.34,7.458333,0.837209
4,22764,57575277,38,9.43,1.22,9.12,1.72,7.427027,0.948718


In [26]:
df_avaliacoes.to_csv('avaliacao.csv', sep=';', index=False)
df_componentes.to_csv('componente.csv', sep=';', index=False)
df_docentes.to_csv('docente.csv', sep=';', index=False)
df_unidades.to_csv('unidade.csv', sep=';', index=False)
df_turmas.to_csv('turma.csv', sep=';', index=False)

In [28]:
import pandas as pd
import requests
import json

api_url = 'http://localhost:8080/'

def make_post(df, endpoint):
    records = json.loads(df.to_json(orient='records'))
    for record in records:
        request = requests.post(endpoint, data=json.dumps(record), headers={'Content-Type': 'application/json'})
        print request.text
        if not request.ok:
            break

def insert(df_url, endpoint):
    df = pd.read_csv(df_url, sep=';')
    make_post(df, endpoint)

if __name__ == '__main__':
    insert('unidade.csv', api_url+'unidade')
    insert('docente.csv', api_url+'docente')
    insert('componente.csv', api_url+'componente')
    insert('turma.csv', api_url+'turma')
    insert('avaliacao.csv', api_url+'avaliacao')

{"lotacao":"DEPARTAMENTO DE INFORMATICA E MATEMATICA APLICADA","id_unidade":125}
{"lotacao":"INSTITUTO METROPOLE DIGITAL","id_unidade":6069}
{"nome":"ADJA FERREIRA DE ANDRADE","formacao":"DOUTORADO","data_admissao":"2012-11-01","id_docente":5755281,"unidade":{"lotacao":null,"id_unidade":6069}}
{"nome":"ANDERSON PAIVA CRUZ","formacao":"DOUTORADO","data_admissao":"2013-04-25","id_docente":5755819,"unidade":{"lotacao":null,"id_unidade":6069}}
{"nome":"ANDRE LUIZ DA SILVA SOLINO","formacao":"MESTRADO","data_admissao":"2016-01-19","id_docente":5757969,"unidade":{"lotacao":null,"id_unidade":6069}}
{"nome":"ANNA GISELLE CAMARA DANTAS RIBEIRO RODRIGUES","formacao":"DOUTORADO","data_admissao":"2017-05-26","id_docente":5759298,"unidade":{"lotacao":null,"id_unidade":6069}}
{"nome":"APUENA VIEIRA GOMES","formacao":"DOUTORADO","data_admissao":"2006-08-25","id_docente":3032089,"unidade":{"lotacao":null,"id_unidade":6069}}
{"nome":"ATHANASIOS TSOUANAS","formacao":"DOUTORADO","data_admissao":"2016-01-

{"nome":"NELIO ALESSANDRO AZEVEDO CACHO","formacao":"DOUTORADO","data_admissao":"2009-01-27","id_docente":5752183,"unidade":{"lotacao":null,"id_unidade":125}}
{"nome":"RAFAEL BESERRA GOMES","formacao":"DOUTORADO","data_admissao":"2014-07-11","id_docente":5756803,"unidade":{"lotacao":null,"id_unidade":125}}
{"nome":"REGIVAN HUGO NUNES SANTIAGO","formacao":"DOUTORADO","data_admissao":"2002-02-19","id_docente":24798,"unidade":{"lotacao":null,"id_unidade":125}}
{"nome":"ROBERTA DE SOUZA COELHO","formacao":"DOUTORADO","data_admissao":"2009-07-01","id_docente":5752649,"unidade":{"lotacao":null,"id_unidade":125}}
{"nome":"SELAN RODRIGUES DOS SANTOS","formacao":"DOUTORADO","data_admissao":"2006-01-09","id_docente":62353,"unidade":{"lotacao":null,"id_unidade":125}}
{"nome":"SILVIA MARIA DINIZ MONTEIRO MAIA","formacao":"DOUTORADO","data_admissao":"2014-02-21","id_docente":5756412,"unidade":{"lotacao":null,"id_unidade":125}}
{"nome":"THAIS VASCONCELOS BATISTA","formacao":"DOUTORADO","data_admissa

{"codigo":"DIM0411","id_componente":21705,"nome":"PROCESSAMENTO DE IMAGENS","unidade":{"lotacao":"DEPARTAMENTO DE INFORMATICA E MATEMATICA APLICADA","id_unidade":125}}
{"codigo":"DIM0052","id_componente":21707,"nome":"LINGUAGEM DE PROGRAMACAO","unidade":{"lotacao":"DEPARTAMENTO DE INFORMATICA E MATEMATICA APLICADA","id_unidade":125}}
{"codigo":"DIM0326","id_componente":21708,"nome":"ALGORITMOS E ESTRUTURAS DE DADOS I","unidade":{"lotacao":"DEPARTAMENTO DE INFORMATICA E MATEMATICA APLICADA","id_unidade":125}}
{"codigo":"DIM0051","id_componente":21709,"nome":"LINGUAGENS FORMAIS E AUTOMATOS","unidade":{"lotacao":"DEPARTAMENTO DE INFORMATICA E MATEMATICA APLICADA","id_unidade":125}}
{"codigo":"DIM0306","id_componente":21710,"nome":"COMPILADORES I","unidade":{"lotacao":"DEPARTAMENTO DE INFORMATICA E MATEMATICA APLICADA","id_unidade":125}}
{"codigo":"DIM0076","id_componente":21711,"nome":"AQUISICAO E REPRESENTACAO DO CONHECIMENTO","unidade":{"lotacao":"DEPARTAMENTO DE INFORMATICA E MATEMATIC

{"codigo":"DIM0337","id_componente":21765,"nome":"COMPUTABILIDADE","unidade":{"lotacao":"DEPARTAMENTO DE INFORMATICA E MATEMATICA APLICADA","id_unidade":125}}
{"codigo":"DIM0331","id_componente":21766,"nome":"MODELOS DE OTIM.SOL.DE PROB.DE EXPLOR.E DIST.DE P.","unidade":{"lotacao":"DEPARTAMENTO DE INFORMATICA E MATEMATICA APLICADA","id_unidade":125}}
{"codigo":"DIM0334","id_componente":21767,"nome":"SISTEMAS DE TEMPO REAL EMBUTIDOS","unidade":{"lotacao":"DEPARTAMENTO DE INFORMATICA E MATEMATICA APLICADA","id_unidade":125}}
{"codigo":"DIM0347","id_componente":21768,"nome":"SISTEMAS MULTIMIDIA","unidade":{"lotacao":"DEPARTAMENTO DE INFORMATICA E MATEMATICA APLICADA","id_unidade":125}}
{"codigo":"DIM0350","id_componente":21769,"nome":"PRATICA EM ENGENHARIA DE SOFTWARE","unidade":{"lotacao":"DEPARTAMENTO DE INFORMATICA E MATEMATICA APLICADA","id_unidade":125}}
{"codigo":"DIM0310","id_componente":21770,"nome":"BANCO DE DADOS NAO CONVENCIONAIS","unidade":{"lotacao":"DEPARTAMENTO DE INFORMATI

{"codigo":"DIM0404","id_componente":23558,"nome":"CALCULO NUMERICO PARA CIENCIA DA COMPUTACAO","unidade":{"lotacao":"DEPARTAMENTO DE INFORMATICA E MATEMATICA APLICADA","id_unidade":125}}
{"codigo":"DIM0305","id_componente":23560,"nome":"PROGRAMACAO MATEMATICA","unidade":{"lotacao":"DEPARTAMENTO DE INFORMATICA E MATEMATICA APLICADA","id_unidade":125}}
{"codigo":"DIM0002","id_componente":24755,"nome":"INTRODUCAO AOS MICROCOMPUTADORES","unidade":{"lotacao":"DEPARTAMENTO DE INFORMATICA E MATEMATICA APLICADA","id_unidade":125}}
{"codigo":"DIM0038","id_componente":24757,"nome":"PROGRAMACAO I","unidade":{"lotacao":"DEPARTAMENTO DE INFORMATICA E MATEMATICA APLICADA","id_unidade":125}}
{"codigo":"DIM0039","id_componente":24758,"nome":"INTRODUCAO AO PROCESSAMENTO DE DADOS","unidade":{"lotacao":"DEPARTAMENTO DE INFORMATICA E MATEMATICA APLICADA","id_unidade":125}}
{"codigo":"DIM0999","id_componente":24959,"nome":"ESTAGIO SUPERVISIONADO","unidade":{"lotacao":"DEPARTAMENTO DE INFORMATICA E MATEMATI

...

{"id":null,"aprovados":0.7518796992,"docente":{"nome":null,"formacao":null,"data_admissao":null,"id_docente":5755939,"unidade":null},"turma":{"ano":null,"nivel":null,"periodo":null,"id_turma":57586295,"componente":null},"qtd_discentes":57,"postura_profissional":9.66,"postura_profissional_DP":0.62,"atuacao_profissional":9.07,"atuacao_profissional_DP":1.38,"media_aprovados":6.196}
{"id":null,"aprovados":0.6440677966,"docente":{"nome":null,"formacao":null,"data_admissao":null,"id_docente":5755939,"unidade":null},"turma":{"ano":null,"nivel":null,"periodo":null,"id_turma":57586296,"componente":null},"qtd_discentes":23,"postura_profissional":9.69,"postura_profissional_DP":0.69,"atuacao_profissional":9.41,"atuacao_profissional_DP":1.25,"media_aprovados":6.3578947368}
{"id":null,"aprovados":0.7142857143,"docente":{"nome":null,"formacao":null,"data_admissao":null,"id_docente":5755939,"unidade":null},"turma":{"ano":null,"nivel":null,"periodo":null,"id_turma":57586297,"componente":null},"qtd_disc

{"id":null,"aprovados":0.7058823529,"docente":{"nome":null,"formacao":null,"data_admissao":null,"id_docente":5756342,"unidade":null},"turma":{"ano":null,"nivel":null,"periodo":null,"id_turma":1245673,"componente":null},"qtd_discentes":5,"postura_profissional":8.68,"postura_profissional_DP":0.98,"atuacao_profissional":8.56,"atuacao_profissional_DP":1.71,"media_aprovados":9.5}
{"id":null,"aprovados":0.9473684211,"docente":{"nome":null,"formacao":null,"data_admissao":null,"id_docente":5756342,"unidade":null},"turma":{"ano":null,"nivel":null,"periodo":null,"id_turma":1245684,"componente":null},"qtd_discentes":6,"postura_profissional":7.62,"postura_profissional_DP":1.88,"atuacao_profissional":8.32,"atuacao_profissional_DP":1.75,"media_aprovados":8.15}
{"id":null,"aprovados":0.8571428571,"docente":{"nome":null,"formacao":null,"data_admissao":null,"id_docente":5756342,"unidade":null},"turma":{"ano":null,"nivel":null,"periodo":null,"id_turma":1245685,"componente":null},"qtd_discentes":6,"postu

{"id":null,"aprovados":0.8108108108,"docente":{"nome":null,"formacao":null,"data_admissao":null,"id_docente":5756592,"unidade":null},"turma":{"ano":null,"nivel":null,"periodo":null,"id_turma":57586679,"componente":null},"qtd_discentes":10,"postura_profissional":9.61,"postura_profissional_DP":0.98,"atuacao_profissional":9.03,"atuacao_profissional_DP":1.57,"media_aprovados":7.33}
{"id":null,"aprovados":0.4951456311,"docente":{"nome":null,"formacao":null,"data_admissao":null,"id_docente":5756802,"unidade":null},"turma":{"ano":null,"nivel":null,"periodo":null,"id_turma":1239543,"componente":null},"qtd_discentes":24,"postura_profissional":9.84,"postura_profissional_DP":0.37,"atuacao_profissional":9.37,"atuacao_profissional_DP":1.09,"media_aprovados":7.4294117647}
{"id":null,"aprovados":0.6388888889,"docente":{"nome":null,"formacao":null,"data_admissao":null,"id_docente":5756802,"unidade":null},"turma":{"ano":null,"nivel":null,"periodo":null,"id_turma":57564842,"componente":null},"qtd_discen

{"id":null,"aprovados":0.3707865169,"docente":{"nome":null,"formacao":null,"data_admissao":null,"id_docente":5757620,"unidade":null},"turma":{"ano":null,"nivel":null,"periodo":null,"id_turma":57576529,"componente":null},"qtd_discentes":48,"postura_profissional":9.61,"postura_profissional_DP":0.74,"atuacao_profissional":9.27,"atuacao_profissional_DP":1.3,"media_aprovados":6.7818181818}
{"id":null,"aprovados":0.72,"docente":{"nome":null,"formacao":null,"data_admissao":null,"id_docente":5757620,"unidade":null},"turma":{"ano":null,"nivel":null,"periodo":null,"id_turma":57576780,"componente":null},"qtd_discentes":13,"postura_profissional":9.78,"postura_profissional_DP":0.55,"atuacao_profissional":9.51,"atuacao_profissional_DP":1.02,"media_aprovados":6.475}
{"id":null,"aprovados":0.4017857143,"docente":{"nome":null,"formacao":null,"data_admissao":null,"id_docente":5757620,"unidade":null},"turma":{"ano":null,"nivel":null,"periodo":null,"id_turma":57586254,"componente":null},"qtd_discentes":20

{"id":null,"aprovados":0.6,"docente":{"nome":null,"formacao":null,"data_admissao":null,"id_docente":5757969,"unidade":null},"turma":{"ano":null,"nivel":null,"periodo":null,"id_turma":57586284,"componente":null},"qtd_discentes":6,"postura_profissional":10.0,"postura_profissional_DP":0.0,"atuacao_profissional":9.13,"atuacao_profissional_DP":0.89,"media_aprovados":6.8333333333}
{"id":null,"aprovados":0.775862069,"docente":{"nome":null,"formacao":null,"data_admissao":null,"id_docente":5757969,"unidade":null},"turma":{"ano":null,"nivel":null,"periodo":null,"id_turma":57587401,"componente":null},"qtd_discentes":16,"postura_profissional":9.89,"postura_profissional_DP":0.31,"atuacao_profissional":9.68,"atuacao_profissional_DP":1.06,"media_aprovados":7.3533333333}
{"id":null,"aprovados":0.7894736842,"docente":{"nome":null,"formacao":null,"data_admissao":null,"id_docente":5757969,"unidade":null},"turma":{"ano":null,"nivel":null,"periodo":null,"id_turma":57597999,"componente":null},"qtd_discentes

In [ ]:
'''
/*Detalhamento de cada DOCENTE*/
SELECT avaliacao.id_docente, nome, AVG(aprovados) AS aprovados, AVG(media_aprovados) AS media_aprovados, AVG(atuacao_profissional) AS atuacao_profissional, AVG(postura_profissional) AS postura_profissional FROM avaliacao LEFT JOIN docente ON avaliacao.id_docente=docente.id_docente GROUP BY avaliacao.id_docente ORDER BY nome;

/*COMPONENTES PELO NOME*/
SELECT * FROM componente_curricular WHERE nome_componente_curricular LIKE '%MATEMATICA%';

/*AVALIACAO PELO NOME DO COMPONENTE*/
SELECT * FROM avaliacao 
JOIN turma ON avaliacao.id_turma=turma.id_turma
JOIN componente_curricular ON turma.id_componente=componente_curricular.id_componente_curricular
JOIN docente ON avaliacao.id_docente=docente.id_docente
WHERE nome_componente_curricular LIKE '%MATEMATICA%';

/*AVALIACOES PELO ID DO COMPONENTE*/
SELECT * FROM avaliacao 
JOIN turma ON avaliacao.id_turma=turma.id_turma
JOIN componente_curricular ON turma.id_componente=componente_curricular.id_componente_curricular
JOIN docente ON avaliacao.id_docente=docente.id_docente
WHERE id_componente_curricular=52653;

/*TURMAS PELO NOME DO COMPONENTE*/
SELECT * FROM componente_curricular JOIN turma ON id_componente_curricular=id_componente WHERE nome_componente_curricular LIKE '%MATEMATICA%';
'''